# MODELS

In [1]:
import pandas as pd
import numpy as np


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier



In [2]:
df = pd.read_csv('X_TRAIN.csv') # need to remove uid and poverty_score_y

In [3]:
test_df = pd.read_csv('X_TEST.csv')

In [4]:
# poverty = poverty['poverty_score_y'].to_numpy()

In [5]:
# forgot about this. for ocd reasons let
df['attended_school'] = df['attended_school'].eq(1)
df = df.drop(columns=['finish_school_age'])

test_df['attended_school'] = test_df['attended_school'].eq(1)
test_df = test_df.drop(columns=['finish_school_age'])


# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Models General case

In [4]:
X = df.drop(columns=['uid', 'poverty_score_y'])
X_test = pd.read_csv('X_TEST.csv')

# Best parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 50}
stacking_subset = df.sample(n=1300, axis=0)
X_stacking = stacking_subset.reset_index()
uids_stacking = X_stacking['uid']
uids_prediction = X_test['uid']
X_stacking = X_stacking.drop(columns=['uid', 'poverty_score_y'])


NameError: name 'df' is not defined

In [ ]:
reasons = ['not_attend_reason1_2',
       'not_attend_reason1_3', 'not_attend_reason1_4', 'not_attend_reason1_5',
       'not_attend_reason1_6', 'not_attend_reason1_7', 'not_attend_reason1_8',
       'not_attend_reason1_9', 'not_attend_reason1_10',
       'not_attend_reason1_11', 'not_attend_reason1_12',
       'not_attend_reason1_13', 'not_attend_reason1_14',
       'not_attend_reason2_2', 'not_attend_reason2_3', 'not_attend_reason2_4',
       'not_attend_reason2_5', 'not_attend_reason2_6', 'not_attend_reason2_7',
       'not_attend_reason2_8', 'not_attend_reason2_9', 'not_attend_reason2_10',
       'not_attend_reason2_11', 'not_attend_reason2_12',
       'not_attend_reason2_13', 'not_attend_reason2_14', 'school_transportation_time']
X = X.drop(columns=reasons)
X_test = X_test.drop(columns=reasons)
X_stacking = X_stacking.drop(columns=reasons)

In [ ]:
new_cols_to_drop = ['uid', 'finish_school_age']
X_test = X_test.drop(columns=new_cols_to_drop)

In [ ]:
X.columns == X_test.columns

In [ ]:
y = df['poverty_score_y'].to_numpy()

### Random Forest**

In [69]:
param_grid = {
    'n_estimators': [10, 50, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X, y)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.2f}")

Best parameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 50}
Best cross-validation accuracy: 0.21


# Do it for reals!

In [ ]:
X_stacking = X_stacking.drop(columns=['index'])

In [ ]:
X_stacking

In [ ]:
# Initialize the Random Forest model
rf = RandomForestClassifier(n_estimators=50, max_depth=10, min_samples_split=5, random_state=42)

# Train the Random Forest model on the training data
rf.fit(X, y)

# Make predictions on the test data
y_stacking = rf.predict_proba(X_stacking)
y_pred = rf.predict_proba(X_test)



In [ ]:
cols = ['subjective_poverty_1', 'subjective_poverty_2', 'subjective_poverty_3', 'subjective_poverty_4', 
        'subjective_poverty_5', 'subjective_poverty_6',
        'subjective_poverty_7', 'subjective_poverty_8', 'subjective_poverty_9', 'subjective_poverty_10']

y_pred = pd.DataFrame(y_pred, columns=cols)
y_stacking = pd.DataFrame(y_stacking, columns=cols)

In [ ]:
pred = pd.concat([uids_prediction, y_pred], axis=1)
pred.rename(columns={'uid': 'psu_hh_idcode'}, inplace=True)

stack = pd.concat([uids_stacking, y_stacking], axis=1)
stack.rename(columns={'uid': 'psu_hh_idcode'}, inplace=True)


In [ ]:
pred.to_csv('testset_predictions.csv', index=False)
stack.to_csv('stacking_predictions.csv', index=False)


# Chenruo's final data
Covariates used:

* role_group
* age_group
* kids_group
* marital_status
* away_group
* mother_education_group
* read_write
* education_group
* preschool_group
* enrollment_grouping

In [ ]:
df = pd.read_csv('chenruos_final.csv')

In [ ]:
y = df['Self_Rank'].to_numpy()
uids = df['Self_id']

In [ ]:
var_used = ['family_role_group', 'age_group', 'kids_group', 'marital_status', 'away_group', 
            'mother_education_group', 'read_write', 'education_group', 'preschool_group', 'enrollment_grouping',
           ]
X = df[var_used]

In [ ]:
X.columns

In [ ]:
dummies = pd.get_dummies(X['family_role_group'], prefix='role', drop_first=True)
X = pd.concat([X, dummies], axis=1)

dummies = pd.get_dummies(X['age_group'], prefix='age', drop_first=True)
X = pd.concat([X, dummies], axis=1)

dummies = pd.get_dummies(X['kids_group'], prefix='role', drop_first=True)
X = pd.concat([X, dummies], axis=1)

dummies = pd.get_dummies(X['marital_status'], prefix='role', drop_first=True)
X = pd.concat([X, dummies], axis=1)

dummies = pd.get_dummies(X['mother_education_group'], prefix='role', drop_first=True)
X = pd.concat([X, dummies], axis=1)

dummies = pd.get_dummies(X['away_group'], prefix='role', drop_first=True)
X = pd.concat([X, dummies], axis=1)

dummies = pd.get_dummies(X['education_group'], prefix='role', drop_first=True)
X = pd.concat([X, dummies], axis=1)

dummies = pd.get_dummies(X['preschool_group'], prefix='role', drop_first=True)
X = pd.concat([X, dummies], axis=1)

dummies = pd.get_dummies(X['enrollment_grouping'], prefix='role', drop_first=True)
X = pd.concat([X, dummies], axis=1)



In [ ]:
X

In [ ]:
old =[ 'family_role_group', 'age_group', 'kids_group', 'marital_status',
       'away_group', 'mother_education_group','education_group',
       'preschool_group', 'enrollment_grouping']
X = X.drop(columns=old)

In [ ]:
X

### Random Forest**

In [ ]:
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X, y)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.2f}")

### kNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=200) ## this says they are all close to the middle

# Perform 5-fold cross-validation
cv_scores = cross_val_score(knn, X, y, cv=5, scoring='accuracy')

# Print the accuracy for each fold
print("Cross-validation scores:", cv_scores)
print(f"Average cross-validation accuracy: {cv_scores.mean():.2f}")



### Logistic

In [ ]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Perform cross-validation with 5 folds
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean accuracy:", cv_scores.mean())

### NN

doesn't converge

In [ ]:
# nn_model = MLPClassifier(max_iter=1000, random_state=42)

# # Define the hyperparameter grid
# param_grid = {
#     'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
#     'activation': ['relu', 'tanh'],
#     'solver': ['adam', 'sgd'],
#     'learning_rate': ['constant', 'adaptive'],
# }

# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=nn_model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# # Perform the grid search
# grid_search.fit(X, y)

# # Print the best parameters and best score
# print("Best parameters:", grid_search.best_params_)
# print("Best cross-validation accuracy:", grid_search.best_score_)
